In [8]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# from MCMAC_pre_merger import MCengine_pre_merger
# from MCMAC_post_merger import MCengine_post_merger 

# change prior: angle 0-45 degree
from modules.MCMAC_pre_merger_changeprior_45degree import MCengine_pre_merger
from modules.MCMAC_post_merger_changeprior_45degree import MCengine_post_merger 


import numpy as np
import pickle

# Input A3376 parameters

In [9]:
pre = False
post = True

change_prior_angle = True

# Setting simulation parameters (define prior distributions)
N_mc = 1000  # Number of Monte Carlo samples

cluster_name ='A3376'

if change_prior_angle is True:
    folder_name = cluster_name + '_0-45degree'
else:
    folder_name = cluster_name

distance_Mpc = 0.97
if distance_Mpc is None:
    arcsecond = 0
redshift = 0.046  # Redshift from the paper

# (mean value, standard deviation)
M1 = (6e14, 1e14)  # Mass distribution of the major cluster
M2 = (1.875e14, 1.125e14)  # Approximated mass distribution of the subcluster
Z1 = (0.046, 0.0001)  # Redshift distribution of A3376 (Subcluster 1)
Z2 = (0.046, 0.0001)  # Redshift distribution of A3376 (Subcluster 2)

TSC_simulation = 0.5  # Time since collision in Gyr

# when you use pre-MCMAC code
if pre is True:
    prefix = "../RealClusters/" + folder_name + "/simulation_pre_output" 

# when you use post-MCMAC code
if post is True:
    prefix = "../RealClusters/" + folder_name + "/simulation_post_output" 

result_file = "result_" + folder_name + ".csv"

## arcsecond to kpc

In [10]:
from astropy.cosmology import Planck18 as cosmo

def arcsecondToKpc(redshift, arcsecond):
    # Convert arcseconds to kiloparsecs
    kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60  # kpc per arcsec
    distance_kpc = arcsecond * kpc_per_arcsec
    return distance_kpc

In [11]:
if distance_Mpc is None and arcsecond:
    distance_Mpc = arcsecondToKpc(redshift, arcsecond) * 0.001 # kpc to Mpc

print(str(distance_Mpc)+ 'Mpc')

D_proj = (distance_Mpc, 0.05*distance_Mpc, 0.05*distance_Mpc)  # Projected separation distance distribution (mean 408 kpc, standard deviation 14 kpc)

0.97Mpc


## Confirm it collision or not

In [12]:
# Execute simulation (derive posterior distributions of dynamical properties such as collision velocity, time since collision, etc.)
if pre is True:
    MCengine_pre_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Probability of system being bound = 1.00

Probability of system being boud is over 50%, then use post merger code.

In [13]:
if post is True:
    MCengine_post_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Completed Monte Carlo iteration 10 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 100 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 1000 of 1000.
~0 minutes remaining


In [14]:
import pickle
import numpy as np
import pandas as pd

if post is True:
    # 単位の定義
    units = {
        'm1': '10^14 M☉',
        'm2': '10^14 M☉',
        'z1': '',
        'z2': '',
        'dproj': 'Mpc',
        'vradobs': 'km s^{-1}',
        'alpha': 'degree',
        'v3dobs': 'km s^{-1}',
        'd3d': 'Mpc',
        'v3dcol': 'km s^{-1}',
        'dmax': 'Mpc',
        'TSM0': 'Gyr',
        'TSM1': 'Gyr',
        'T': 'Gyr',
        'prob': ''
    }

    # パラメータファイルのリスト
    parameters = ['_m_1.pickle', '_m_2.pickle', '_z_1.pickle', '_z_2.pickle', '_d_proj.pickle', 
              '_v_rad_obs.pickle', '_alpha.pickle', '_v_3d_obs.pickle', '_d_3d.pickle', 
              '_v_3d_col.pickle', '_d_max.pickle', '_TSM_0.pickle', '_TSM_1.pickle', 
              '_T.pickle', '_prob.pickle']

    results = []

    for param in parameters:
        with open(prefix + param, 'rb') as f:
            data = pickle.load(f)
            if param in ['_m_1.pickle', '_m_2.pickle']:
                # 質量をe14単位に変換
                data = np.array(data) / 1e14
            
            # Medianを計算
            median = np.median(data)
            
            # 68%および95%の信頼区間を計算
            lcl_68, ucl_68 = np.percentile(data, [16, 84])
            lcl_95, ucl_95 = np.percentile(data, [2.5, 97.5])
            
            param_name = param.replace('_', '').replace('.pickle', '')
            
            # 数値を有効数字2桁にフォーマット
            if param_name not in ['z_1', 'z_2']:
                median = round(median, 2)
                lcl_68 = round(lcl_68, 2)
                ucl_68 = round(ucl_68, 2)
                lcl_95 = round(lcl_95, 2)
                ucl_95 = round(ucl_95, 2)
            
            results.append({
                'Parameter': param_name,
                'Units': units[param_name],
                'Median': median,
                '68% LCL': lcl_68,
                '68% UCL': ucl_68,
                # '95% LCL': lcl_95,
                # '95% UCL': ucl_95
            })

    # 結果をDataFrameに変換
    df_results = pd.DataFrame(results)
    display(df_results)

    # TSM0とTSM1に対応する行の抽出
    tsm0_row = df_results[df_results['Parameter'] == 'TSM0']
    tsm1_row = df_results[df_results['Parameter'] == 'TSM1']
    print("tsm0_row", tsm0_row)
    print("tsm1_row", tsm1_row)
    
    # CSVに出力するための新しい行を作成
    new_row = {
        'ClusterName': cluster_name,
        'MCMAC_TSC0': tsm0_row['Median'].values[0],
        'MCMAC_TSC0.lower': tsm0_row['68% LCL'].values[0],
        'MCMAC_TSC0.upper': tsm0_row['68% UCL'].values[0],
        'MCMAC_TSC1': tsm1_row['Median'].values[0],
        'MCMAC_TSC1.lower': tsm1_row['68% LCL'].values[0],
        'MCMAC_TSC1.upper': tsm1_row['68% UCL'].values[0],
        'Simulation_TSC': TSC_simulation
    }

    # 新しい行をDataFrameに変換
    new_row_df = pd.DataFrame([new_row])

    # CSVファイルに出力
    new_row_df.to_csv(result_file, index=False)


,Parameter,Units,Median,68% LCL,68% UCL
0,m1,10^14 M☉,6.03,5.05,7.01
1,m2,10^14 M☉,1.97,1.00,3.02
2,z1,,0.05,0.05,0.05
3,z2,,0.05,0.05,0.05
4,dproj,Mpc,0.97,0.93,1.02
5,vradobs,km s^{-1},23.79,7.42,53.80
6,alpha,degree,5.22,1.57,10.29
7,v3dobs,km s^{-1},315.80,83.21,960.69
8,d3d,Mpc,0.98,0.94,1.02
9,v3dcol,km s^{-1},1797.32,1700.82,2027.46


tsm0_row    Parameter Units  Median  68% LCL  68% UCL
11      TSM0   Gyr    0.88     0.63      1.0
tsm1_row    Parameter Units  Median  68% LCL  68% UCL
12      TSM1   Gyr    1.25     1.02     2.01
